## Prepreatory Steps

Will comment as needed (and in general please assume that). Especially as this is not my first work...

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv('../data/train_data.csv')
print(df.shape)
df.head()

(100, 13)


,Person ID,Gender,Age,Occupation,Sleep Duration (hours),Quality of Sleep (scale: 1-10),Physical Activity Level (minutes/day),Stress Level (scale: 1-10),BMI Category,Blood Pressure (systolic/diastolic),Heart Rate (bpm),Daily Steps,Sleep Disorder
0,d29d53701d3c859e29e1b90028eec1ca8e2f29439198b6...,Male,62,Student,5.802690,9.684249,26.361925,9.247187,Overweight,131/78,88.434966,12135.856393,No Disorder
1,71a1c003a2b855d85582c8f6c7648c49d3fe836408a7e1...,Female,35,Manual Labor,7.928079,4.589872,111.302978,3.092288,Overweight,117/69,62.090257,19871.474804,Insomnia
2,86e50149658661312a9e0b35558d84f6c6d3da797f552a...,Female,50,Office Worker,4.425595,6.809697,28.559919,5.168146,Obese,135/86,94.422789,18754.661163,No Disorder
3,093434a3ee9e0a010bb2c2aae06c2614dd24894062a1ca...,Male,38,Office Worker,9.031985,4.682931,76.911892,6.037287,Overweight,119/69,80.149192,12658.283618,No Disorder
4,e3d6c4d4599e00882384ca981ee287ed961fa5f3828e2a...,Female,34,Student,8.719110,6.729776,11.742485,2.029544,Normal,117/69,74.926656,17742.211473,Insomnia


In [10]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')

df.columns

Index(['person_id', 'gender', 'age', 'occupation', 'sleep_duration_(hours)',
       'quality_of_sleep_(scale:_1-10)',
       'physical_activity_level_(minutes/day)', 'stress_level_(scale:_1-10)',
       'bmi_category', 'blood_pressure_(systolic/diastolic)',
       'heart_rate_(bpm)', 'daily_steps', 'sleep_disorder'],
      dtype='object')

In [11]:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()])

Phew, no nulls.


In [12]:
#Unfortunately can't do my typical drop, at least at the moment, as look at how granular sleep duration is... hence per my coding
#it'll get dropped... and unfortunately a type check wouldn't work as personid could be coded to be an integer...

# exceptions = ['id']

# for i in df.columns:
#     if len(df[i].unique()) == df.shape[0] and i not in exceptions:
#         df = df.drop([i], axis=1) #Not adding anything to our model... Hmm, now that I think about it, let's make this better:
#         print(f"Just dropped {i} as it wasn't adding any value whatsoever to our model.")
#     else:
#         print(f"Well, {i} has {len(df[i].unique())} unique fields, so we'll keep it...for now!")

In [13]:
df.drop('person_id', axis=1, inplace=True)

## Initial EDA

In [ ]:
i = -1

In [15]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 0.


gender
Female    0.53
Male      0.47
Name: proportion, dtype: float64

In [19]:
#Guess we'll make female, the baseline, be 0:
df[df.columns[i]] = [0 if value == 'Female' else 1 for value in df[df.columns[i]]]

#Pretty even split on the data though. An obvious interest will be if fe/males of a certain age are more likely towards whatever
#Ie might do some banding later.

In [22]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)[:5]

Now looking at feature 1.


age
47    0.07
29    0.05
18    0.05
39    0.05
42    0.05
Name: proportion, dtype: float64

In [23]:
df[df.columns[i]].describe()

#Pretty nice distrubtion re the means and medians. Decent sized std, but likely tbe given the smaller size. Plus, the population
#for this study has whatever conditions in the first place, so given that we have younger people...

#I mean, maybe not. Ie their information was definitely recorded, but perhaps at random from a regular therapy and it's not like
#they claimed (neccesarily) that they had issues. Regardless...

count    100.000000
mean      41.240000
std       14.575557
min       18.000000
25%       29.000000
50%       41.000000
75%       50.000000
max       90.000000
Name: age, dtype: float64

In [24]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 2.


occupation
Student          0.28
Manual Labor     0.28
Office Worker    0.26
Retired          0.18
Name: proportion, dtype: float64

In [27]:
#Likely the students are the younglings and the retired are the oldies, hence a bit of a redundancy. However, at least the
#middle two might be of use (re. predictng conditions).

#Likely there's what to rationialize about ach of the 4 occupations, however for now let us actually do data.

print(df[df['occupation']=='Student']['age'].describe())

print(df[df['occupation']=='Retired']['age'].describe())

#Oh, this is quite interesting - the student's group retained an incredibly close mean and median of the general train population!
#Even the std is quite the same... and notice the extreme 90 is listed as a student.... Him aside would still lead to this
#results which I would not at all expect.

#Similarily, retired also has a nice mix - similar enough mean and medium and std. However, the extremes seem peculiar to me -
#we have an 18 year old that's retired?? Furthermore, the max age is only 71?? Note momentarily that only 3 people are above 70
#in the train data - him, this mysterious 90 year old student, and another that works in manual labour. I wonder how compatabile
#the model will be above 70, but regardless... So too re. predicting younglings.

count    28.000000
mean     42.678571
std      15.869966
min      18.000000
25%      32.500000
50%      40.000000
75%      49.250000
max      90.000000
Name: age, dtype: float64
count    18.000000
mean     43.055556
std      16.307603
min      18.000000
25%      29.000000
50%      44.000000
75%      51.750000
max      71.000000
Name: age, dtype: float64


In [30]:
df[df['age']>70]

,gender,age,occupation,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),bmi_category,blood_pressure_(systolic/diastolic),heart_rate_(bpm),daily_steps,sleep_disorder
14,1,71,Retired,10.359016,4.810883,90.792271,8.962384,Obese,145/93,54.390502,18518.998249,No Disorder
33,1,90,Student,10.544222,5.072626,86.756934,6.157313,Normal,145/96,70.477940,16794.322582,Sleep Apnea
44,1,72,Manual Labor,9.119520,9.822810,34.945149,0.986734,Overweight,136/86,67.302059,12449.886314,No Disorder


In [32]:
df[df['age']<19]

#On the topic I wanted to see the younglings:

#5, so a bit more represenaton (and this is just the minimum) - still have 19s...+ With most on the social side being
#anything but retired. Let us continue.

,gender,age,occupation,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),bmi_category,blood_pressure_(systolic/diastolic),heart_rate_(bpm),daily_steps,sleep_disorder
5,1,18,Manual Labor,6.279765,8.170468,115.974793,9.277920,Obese,119/66,53.056695,11688.879016,No Disorder
54,1,18,Retired,9.434025,5.121776,89.981710,5.040794,Underweight,109/60,86.032812,10940.364722,Insomnia
70,0,18,Student,10.137916,6.420820,93.780746,6.710281,Normal,109/60,65.685087,2803.351228,No Disorder
86,1,18,Office Worker,9.354844,4.416373,29.882018,3.140683,Overweight,109/60,76.753739,15885.086170,No Disorder
96,0,18,Student,4.749350,6.592543,80.587240,5.210378,Underweight,109/61,73.842151,12461.582036,Insomnia


In [36]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]] = round(df[df.columns[i]], 3)  #I'll accept the slight loss in accuracy... way too annoying to look at
#otherwise. I did 3 as eh I'm comfortable by default with 3 decimal places.
df[df.columns[i]].describe()

Now looking at feature 3.


count    100.000000
mean       8.038980
std        2.143283
min        4.100000
25%        6.247500
50%        8.392500
75%        9.641250
max       12.056000
Name: sleep_duration_(hours), dtype: float64

Our first feature deserving of its own markdown as I suspect it's that important:

Slightly to the left with a nice standard deviation and obvious interest on both extremes, especially in the context of our problem!

However, what is sleep? Is this uninterupted? Was this averaged for each person for a month (seemingly something like that given the original decimal places...)? Furthermore, is this assuming a circadian rhythm without counting 'catnaps' and the like? What about coffee? What about how effctive they are during the day (potentially gauged via other activities)?

In [37]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]] = round(df[df.columns[i]], 3)
df[df.columns[i]].describe()

#Unsure the scale and likely very subjective, however we at least have a nice enough sample size to be comfortable with such maters
#So too such an idea for the other similar columns.

#Likely somewhat dependent upon the previous feature, as I'm assuming the vast majority of people need more-or-less the same amount
#of sleep. Ie the probability that A's sleep is doubly effective as B's is remote.

Now looking at feature 4.


count    100.000000
mean       6.010290
std        2.004661
min        0.952000
25%        4.694250
50%        6.069000
75%        7.086750
max       10.197000
Name: quality_of_sleep_(scale:_1-10), dtype: float64

In [38]:
#Uhhh.... how do we have the following though:
df[df['quality_of_sleep_(scale:_1-10)']>10]

,gender,age,occupation,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),bmi_category,blood_pressure_(systolic/diastolic),heart_rate_(bpm),daily_steps,sleep_disorder
39,1,52,Retired,9.612,10.197,112.403821,8.871093,Obese,136/87,76.468136,6327.733960,Insomnia
80,1,29,Office Worker,8.488,10.181,19.723080,10.280682,Obese,124/72,55.307289,6805.765101,No Disorder
85,1,51,Student,7.558,10.110,32.962618,5.233295,Overweight,125/81,87.378315,15008.085104,No Disorder


In [40]:
#Thanks to ChatGPT re. the syntax:
df['quality_of_sleep_(scale:_1-10)'] = np.where(
    df['quality_of_sleep_(scale:_1-10)'] > 10, 10, df['quality_of_sleep_(scale:_1-10)']
)

In [42]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]] = round(df[df.columns[i]], 3)
df[df.columns[i]].describe()

Now looking at feature 5.


count    100.000000
mean      67.295760
std       34.881286
min        9.584000
25%       34.333000
50%       68.774500
75%       98.309750
max      121.595000
Name: physical_activity_level_(minutes/day), dtype: float64

In [46]:
#Just curious:
df[df['physical_activity_level_(minutes/day)']>67][['age', 'sleep_duration_(hours)']].describe()
#Age is practically same, yet slightly more sleep. Which makes sense practically. 

,age,sleep_duration_(hours)
count,52.000000,52.000000
mean,41.730769,8.181904
std,15.759361,2.167899
min,18.000000,4.100000
25%,29.750000,6.397000
50%,41.500000,8.552000
75%,50.500000,9.656750
max,90.000000,12.056000


In [48]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]] = round(df[df.columns[i]], 3)
df[df.columns[i]].describe()

Now looking at feature 6.


count    100.000000
mean       5.750550
std        3.079122
min        0.952000
25%        3.064250
50%        5.744000
75%        8.962750
max       10.420000
Name: stress_level_(scale:_1-10), dtype: float64

In [53]:
#Gah, same thing...
print(df[df['stress_level_(scale:_1-10)']>10].shape[0])

9


In [54]:
df['stress_level_(scale:_1-10)'] = np.where(
    df['stress_level_(scale:_1-10)'] > 10, 10, df['stress_level_(scale:_1-10)']
)

0

In [55]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 7.


bmi_category
Overweight     0.28
Underweight    0.27
Obese          0.24
Normal         0.21
Name: proportion, dtype: float64

In [56]:
df = pd.get_dummies(df, columns=['bmi_category'], dtype=int, drop_first=True)
df.head()

,gender,age,occupation,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),blood_pressure_(systolic/diastolic),heart_rate_(bpm),daily_steps,sleep_disorder,bmi_category_Obese,bmi_category_Overweight,bmi_category_Underweight
0,1,62,Student,5.803,9.684,26.362,9.247,131/78,88.434966,12135.856393,No Disorder,0,1,0
1,0,35,Manual Labor,7.928,4.590,111.303,3.092,117/69,62.090257,19871.474804,Insomnia,0,1,0
2,0,50,Office Worker,4.426,6.810,28.560,5.168,135/86,94.422789,18754.661163,No Disorder,1,0,0
3,1,38,Office Worker,9.032,4.683,76.912,6.037,119/69,80.149192,12658.283618,No Disorder,0,1,0
4,0,34,Student,8.719,6.730,11.742,2.030,117/69,74.926656,17742.211473,Insomnia,0,0,0


In [ ]:
stopstopstop

#Start here next time...

In [ ]:
i = 7 #Getting to it later. Hence changing ti to this
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

In [62]:
i = 8
print(f"Now looking at feature {i}.")
df[df.columns[i]] = round(df[df.columns[i]], 3)
df[df.columns[i]].describe()

Now looking at feature 8.


count    100.000000
mean      75.483120
std       15.091807
min       49.291000
25%       62.206250
50%       75.859500
75%       88.796000
max      102.046000
Name: heart_rate_(bpm), dtype: float64

In [61]:
i = 9
print(f"Now looking at feature {i}.")
df[df.columns[i]] = round(df[df.columns[i]], 3)
df[df.columns[i]].describe()

#Hmm, potentially related to the exercise we saw earlier. On that note - blood pressure, stress as well.

Now looking at feature 9.


count      100.000000
mean     11112.861560
std       5123.171202
min       2066.203000
25%       6313.406750
50%      11614.922500
75%      15044.992250
max      19871.475000
Name: daily_steps, dtype: float64

In [63]:
i = 10
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

#Hmmm..... Let's see if the data is much different between people with insomnia vs. sleep apnea, then we can simplify
#matters to just no/yes disorder. especially given our small sample size I'm inclined to do so.

Now looking at feature 10.


sleep_disorder
No Disorder    0.70
Insomnia       0.23
Sleep Apnea    0.07
Name: proportion, dtype: float64

In [65]:
df[df['sleep_disorder']=='Insomnia'].describe()

,gender,age,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),heart_rate_(bpm),daily_steps,bmi_category_Obese,bmi_category_Overweight,bmi_category_Underweight
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,0.391304,41.391304,7.982913,6.434304,70.058217,5.079174,70.499000,12292.983043,0.260870,0.086957,0.347826
std,0.499011,13.107039,2.280035,1.826256,36.607055,2.685788,14.062663,4888.651326,0.448978,0.288104,0.486985
min,0.000000,18.000000,4.501000,2.902000,10.893000,1.001000,51.192000,3186.612000,0.000000,0.000000,0.000000
25%,0.000000,34.500000,6.018000,5.307000,41.304000,3.062000,58.199500,10090.489000,0.000000,0.000000,0.000000
50%,0.000000,43.000000,7.928000,6.272000,73.492000,4.799000,72.523000,12461.582000,0.000000,0.000000,0.000000
75%,1.000000,50.500000,9.626000,7.402500,104.648500,7.611000,79.887000,15747.895000,0.500000,0.000000,1.000000
max,1.000000,61.000000,11.513000,10.000000,114.899000,9.872000,99.371000,19871.475000,1.000000,1.000000,1.000000


In [66]:
df[df['sleep_disorder']=='Sleep Apnea'].describe()

,gender,age,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),heart_rate_(bpm),daily_steps,bmi_category_Obese,bmi_category_Overweight,bmi_category_Underweight
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.000000
mean,0.285714,41.428571,8.687286,6.231571,66.940143,6.651429,76.753714,10764.174429,0.0,0.285714,0.428571
std,0.487950,23.251216,2.999533,1.256492,21.785154,2.601644,11.739391,4532.309015,0.0,0.487950,0.534522
min,0.000000,22.000000,4.362000,4.368000,34.340000,2.853000,62.201000,4537.846000,0.0,0.000000,0.000000
25%,0.000000,26.500000,6.859000,5.521500,51.030000,5.064500,67.401000,7852.524000,0.0,0.000000,0.000000
50%,0.000000,36.000000,10.367000,6.339000,75.179000,6.729000,79.118000,9843.407000,0.0,0.000000,0.000000
75%,0.500000,44.500000,10.726500,6.850000,84.248500,8.424500,83.214000,14234.298500,0.0,0.500000,1.000000
max,1.000000,90.000000,10.911000,8.171000,88.505000,10.000000,94.727000,16794.323000,0.0,1.000000,1.000000


At least from a quick overview:

More males are insomnias.

Age is roughly the same. Much higher median by insomnia though, however to be expected given the counts...

Sleep is interesting - while insomnias are 8 each, apneacs are 8.7 and a hefty 10.3 for median! So, depending upon matters might tread more carefully.

Quality and physical activity is roughly the same.

Oooh, stress - about 1.6 and 2 units higher than the insomniac! Potentially correlatd but a bit higher heart rate too.

Hmm, and htte insomniac takes a lot more steps. Interesting though as they sleep less so in a way makes sense...

Nothing is really jumping out at me re the bmis.

In [68]:
#Fine, let's look at default then call it fo the day:

df[df['sleep_disorder']=='No Disorder'].describe()

,gender,age,sleep_duration_(hours),quality_of_sleep_(scale:_1-10),physical_activity_level_(minutes/day),stress_level_(scale:_1-10),heart_rate_(bpm),daily_steps,bmi_category_Obese,bmi_category_Overweight,bmi_category_Underweight
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,0.514286,41.171429,7.992571,5.841871,66.423657,5.852100,76.993700,10759.976071,0.257143,0.342857,0.228571
std,0.503405,14.248286,2.026119,2.100930,35.690868,3.195778,15.523966,5258.277917,0.440215,0.478091,0.422944
min,0.000000,18.000000,4.100000,0.952000,9.584000,0.952000,49.291000,2066.203000,0.000000,0.000000,0.000000
25%,0.000000,29.000000,6.445750,4.437250,32.343500,3.010500,63.559500,6110.149250,0.000000,0.000000,0.000000
50%,1.000000,39.500000,8.207500,5.859500,67.226000,5.965000,77.869500,11028.510500,0.000000,0.000000,0.000000
75%,1.000000,50.000000,9.526250,7.152250,96.690500,9.196500,90.754500,14917.918250,0.750000,1.000000,0.000000
max,1.000000,72.000000,12.056000,10.000000,121.595000,10.000000,102.046000,19581.377000,1.000000,1.000000,1.000000


I'll focus on key take aways (differeces):

Lolololoolool quality of sleep is lower?!?!

Averaged stress between the two, so that'd be a bit mre difficult if we get rid of.

Similar heart rate to sleep apneas.

Hmm, so it might be an issue. I think I'll look at correlations then do the ttests.

In [60]:
df[df['bmi_category_Underweight']==1]['sleep_disorder'].value_counts(normalize=True)

sleep_disorder
No Disorder    0.592593
Insomnia       0.296296
Sleep Apnea    0.111111
Name: proportion, dtype: float64

In [58]:
df['sleep_disorder'].value_counts(normalize=True)

sleep_disorder
No Disorder    0.70
Insomnia       0.23
Sleep Apnea    0.07
Name: proportion, dtype: float64